# OneStopEnglish: Data Cleaning and EDA
This script imports the One Stop English corpus, which consists of leveled text entries (beginner, intermediate, advanced) aligned to the same topic. Several features of the text entries are calculated using the `textstat` and `spacy` packages. These numerical features are then exported to a csv so they can be incoporated in a machine learning classification model that evaluates text complexity. Additional csv files are exported to finetune simpleGPT2 models for text generation. A list of the data prepared in this script is included below:
1. One column CSV files containing only text entries to be used to train NLP (simpleGPT2) models (**NOTE**, these can be created at the "sentence" and "passage" levels):
    1. _OSE\_adv\_int_: Matched advanced and intermediate texts, difficulty separated by indicator.
    1. _OSE\_adv\_ele_: Matched advanced and elementary texts, difficulty separated by indicator.
    1. _OSE\_int\_ele_: Matched intermediate adn elementary texts, difficulty separated by indicator.
1. A dataframe containing numerical characteristics about text entries to be used in training ML models to evaluate text complexity.
    1. _OSE\_TextFeat_

In [ ]:
# Import packages
import numpy as np
import pandas as pd
import spacy
import nltk
import re
import textstat
import os
from os import listdir
from os.path import join, isfile
import chardet

In [ ]:
# Function splits text, t, into sentences
def sent_break(t):
    nlp = spacy.load('en_core_web_sm')
    doc = nlp(t)
    return doc.sents

# Function counts the number of words in text t
def word_count(t):
    sents = sent_break(t)
    n_words = 0
    for s in sents:
        n_words += len([token for token in s])
    return n_words

# Function counts the number of sentences in text t
def sent_count(t):
    sents = sent_break(t)
    return len(list(sents))

In [ ]:
# Sets option to display all text in pandas dataframes
pd.set_option('display.max_colwidth', None)

# Change working directory to main project directory
os.chdir("..")

# Data Type: Either "TextByReadingLevel" or "SentenceAligned"
DatType = 'TextByReadingLevel/'

# Get file path for OneStopEnglish Corpus from individual TXT files.
RawDat = os.getcwd() + '/data_raw/OneStopEng/' + DatType

# Set location to save clean data
ClnDat = os.getcwd() + '/data_clean/'

In [ ]:
# For some reason there are two "Int" files--do not import the first, as it contains errors
SubDir = [join(RawDat, f)+'/' for f in listdir(RawDat) if f != 'Int-Txt']

# Load all texts into pandas data frame
TextDF = pd.DataFrame()

for s in SubDir:

    TextFiles = listdir(s)

    for t in TextFiles:
        # There is a hidden .ds_store file that should be skipped when importing data
        if t == '.DS_Store':
            pass
        else:
            with open(s + t, 'rb') as rawdat:
                result = chardet.detect(rawdat.read(1000))

                
            file = open(s + t, 'r', encoding = result['encoding'])
            text = file.read()
            file.close()

            # Append text data and file name to dataframe
            TextDF = TextDF.append({'file_nm' : t,
                                    'text' : text},
                                   ignore_index = True)

In [ ]:
# Isolate name and level of text
TextDF['name'] = TextDF['file_nm'].str[:-8]
TextDF['level'] = TextDF['file_nm'].str[-7:-4]
TextDF['text'] = TextDF['text'].apply(str)

# TextDF.groupby(['level'])['text'].count()

# Intermediate text is labeled--this should be removed
# TextDF['text'] = TextDF['text'].map(lambda x: x.lstrip('Intermediate'))
# TextDF['text'] = TextDF['text'].str.replace('\n', ' ')

In [ ]:
# Pivot text to level
TextDF_w = TextDF.pivot(index = ['name'], columns = ['level'], values = ['text']).reset_index()

# Flatten column index of pivot table
TextDF_w.columns = TextDF_w.columns.map('_'.join).str.strip('_')

# Match text pairs accross levels to train GPT2 Model
TextDF_w['adv_int'] = TextDF_w['text_adv'] + '\n |<endoflevelone>| \n' + TextDF_w['text_int']
TextDF_w['adv_ele'] = TextDF_w['text_adv'] + '\n |<endoflevelone>| \n' + TextDF_w['text_ele']
TextDF_w['int_ele'] = TextDF_w['text_int'] + '\n |<endoflevelone>| \n' + TextDF_w['text_ele']

# Export matched text columns to csv file to finetune SimpleGPT2, need to specify encoding or odd figures are saved
# One column must be included for file to successfully be read into SimpleGPT2
for lev in ['adv_int', 'adv_ele', 'int_ele']:
    TextDF_w[lev] = TextDF_w[lev].apply(str)
    TextDF_w[lev].to_csv(ClnDat+'OSE_'+lev+'.csv', encoding = 'utf-8-sig', index = False)

In [ ]:
# Clean text (text_c) by lowering, removing numbers, removing symbols, and reducing to single space
TextDF['text_c'] = TextDF['text'].str.lower().str.replace(r'[^a-zA-Z\s]', '')
TextDF['text_c'] = TextDF['text_c'].str.replace(r'\n', ' ').str.replace(r'\s+\s', ' ')

In [ ]:
# This step is separate from the other text statistic computations because it takes a longer time to run
# Compute text difficulty using textstat
TextDF['difficulty'] = TextDF['text'].apply(textstat.flesch_reading_ease)

# Compute sentence length
TextDF['n_sent'] = TextDF['text'].apply(sent_count)

In [ ]:
# Compute additional text statistics
TextDF['n_syll'] = TextDF['text'].apply(textstat.syllable_count) # Number of syllables
TextDF['n_lex'] = TextDF['text'].apply(textstat.lexicon_count) # Number of words
TextDF['lex_sent'] = TextDF['n_lex']/TextDF['n_sent'] # Word to sentence ratio
TextDF['syll_lex'] = TextDF['n_syll']/TextDF['n_lex'] # Syllable to word ratio

# Save text statistc data to csv
TextDF.to_csv(ClnDat+'OSE_TextFeat.csv', encoding = 'utf-8-sig', index = False)

In [ ]:
TextDF.groupby('level')['n_sent'].plot(kind = 'hist', legend = True, alpha = 0.5, 
                                       title = 'Number of Sentences by Text Level')

In [ ]:
TextDF.groupby('level')['n_syll'].plot(kind = 'hist', legend = True, alpha = 0.5, 
                                       title = 'Number of Syllables by Text Level')

In [ ]:
TextDF.groupby('level')['n_lex'].plot(kind = 'hist', legend = True, alpha = 0.5,
                                      title = 'Number of Words by Text Level')

In [ ]:
TextDF.groupby('level')['lex_sent'].plot(kind = 'hist', legend = True, alpha = 0.5,
                                         title = 'Word to Sentence Ratio by Text Level')

In [ ]:
TextDF.groupby('level')['syll_lex'].plot(kind = 'hist', legend = True, alpha = 0.5, 
                                         title = 'Syllable to Word Ratio by Text Level')

In [ ]:
TextDF.groupby('level')['difficulty'].plot(kind = 'hist', legend = True, alpha = 0.5, 
                                           title = 'Readability Score by Text Level')

In [ ]:
# Create cleaned sentence aligned files for text generation analysis

# Data Type: Either "TextByReadingLevel" or "SentenceAligned"
DatType = 'SentenceAligned/'

# Get file path for OneStopEnglish Corpus from individual TXT files.
RawDat = os.getcwd() + '/data_raw/OneStopEng/' + DatType

t_files = ['ADV-ELE.txt', 'ADV-INT.txt', 'ELE-INT.txt']
t = 'ADV-ELE.txt'

l1 = 'adv'
l2 = 'ele'
with open(RawDat + t, 'rb') as rawdat:
    result = chardet.detect(rawdat.read(10000))
    
file = open(RawDat + t, 'r', encoding = result['encoding'])
text = file.read()
file.close()

text = text.split('\n')

TextDF = pd.DataFrame(text, columns = ['text'])
TextDF = TextDF[~TextDF['text'].isin(['*******', '', ' '])].reset_index().drop(columns = 'index')
TextDF['level'] = [l1, l2]*int(len(TextDF.index)/2)
TextDF['label'] = list(np.repeat(list(range(1, int(len(TextDF.index)/2)+1)), 2))
TextDF['label'] = 's' + TextDF['label'].apply(str)
TextDF = TextDF.pivot(index = ['label'], columns = ['level'], values = ['text']).reset_index()
TextDF

In [ ]:
DatType = 'SentenceAligned/'

# Get file path for OneStopEnglish Corpus from individual TXT files.
RawDat = os.getcwd() + '/data_raw/OneStopEng/' + DatType

t_files = ['ADV-ELE.txt', 'ADV-INT.txt', 'ELE-INT.txt']

for t in t_files:
        
    # Create labels depending on which file inported
    if t == 'ADV-ELE.txt':
        l1 = 'adv'
        l2 = 'ele'
    if t == 'ADV-INT.txt':
        l1 = 'adv'
        l2 = 'int'
    if t == 'ELE-INT.txt':
        l1 = 'ele'
        l2 = 'int'
        
    with open(RawDat + t, 'rb') as rawdat:
        result = chardet.detect(rawdat.read(10000))

    file = open(RawDat + t, 'r', encoding = result['encoding'])
    text = file.read()
    file.close()

    text = text.split('\n')

    TextDF = pd.DataFrame(text, columns = ['text'])
    TextDF = TextDF[~TextDF['text'].isin(['*******', '', ' '])].reset_index().drop(columns = 'index')
    TextDF['level'] = [l1, l2]*int(len(TextDF.index)/2)
    TextDF['label'] = list(np.repeat(list(range(1, int(len(TextDF.index)/2)+1)), 2))
    TextDF['label'] = 's' + TextDF['label'].apply(str)

    # Pivot text to level
    TextDF = TextDF.pivot(index = ['label'], columns = ['level'], values = ['text']).reset_index()

    # Flatten column index of pivot table
    TextDF.columns = TextDF.columns.map('_'.join).str.strip('_')
    
    # Create text variable in which more difficult sentence is above easier sentence, separated by |<endoflevelone>|
    if t == 'ELE-INT.txt':
        l1 = 'int'
        l2 = 'ele'
    else:
        pass
    
    NewVar = l1+'_'+l2
    TextDF[NewVar] = TextDF['text_'+l1] + '\n |<endoflevelone>| \n' + TextDF['text_'+l2]
    TextDF = TextDF.drop_duplicates()
    
    # Export TextDF to csv file
    TextDF[NewVar].to_csv(ClnDat+'OSE_'+NewVar+'_sent.csv', encoding = 'utf-8-sig', index = False)

In [ ]:
# DO NOT USE--IMPORTS TEXT FILE USING PANDAS, AND SOME ROWS ARE MISALIGNED
# THERE IS ALSO AN ISSUE LOADING THE 'ELE-INT' FILE USING PANDAS

# Create cleaned sentence aligned files for text generation analysis

# Data Type: Either "TextByReadingLevel" or "SentenceAligned"
DatType = 'SentenceAligned/'

# Get file path for OneStopEnglish Corpus from individual TXT files.
RawDat = os.getcwd() + '/data_raw/OneStopEng/' + DatType

t_files = ['ADV-ELE.txt', 'ADV-INT.txt', 'ELE-INT.txt']

for t in t_files:
    
    # Read in txt file
    TextDF = pd.read_csv(RawDat + t, sep = '\n', header = None)
    
    # Create labels depending on which file inported
    if t == 'ADV-ELE.txt':
        l1 = 'adv'
        l2 = 'ele'
    if t == 'ADV-INT.txt':
        l1 = 'adv'
        l2 = 'int'
    if t == 'ELE-INT.txt':
        l1 = 'ele'
        l2 = 'int'
        
    TextDF.columns = ['text']
    TextDF['level'] = [l1, l2, 'split']*int(len(TextDF.index)/3)
    TextDF['label'] = list(np.repeat(list(range(1, int(len(TextDF.index)/3)+1)), 3))
    TextDF = TextDF[TextDF['level'] != 'split'].reset_index().drop(columns = ['index'])
    TextDF['label'] = 's' + TextDF['label'].apply(str)

    # Pivot text to level
    TextDF = TextDF.pivot(index = ['label'], columns = ['level'], values = ['text']).reset_index()

    # Flatten column index of pivot table
    TextDF.columns = TextDF.columns.map('_'.join).str.strip('_')
    
    # Create text variable in which more difficult sentence is above easier sentence, separated by |<endoflevelone>|
    if t == 'ELE-INT.txt':
        l1 = 'int'
        l2 = 'ele'
    else:
        pass
    
    NewVar = l1+'_'+l2
    TextDF[NewVar] = TextDF['text_'+l1] + '\n |<endoflevelone>| \n' + TextDF['text_'+l2]
    
    # Export TextDF to csv file
    TextDF[NewVar].to_csv(ClnDat+'OSE_'+NewVar+'_sent.csv', encoding = 'utf-8-sig', index = False)

In [ ]:
###################################
# IMPORT DATA STORED IN CSV FILES #
###################################

# Get file path to OneStopEnglish Corpus including combined CSV files.
FilePath = os.getcwd() + '/OneStopEng/TextOneCSVPerFile/'

# Texts are organized by difficulty level
Files = listdir(FilePath)

# Import all texts into pandas dataframe
oseDF = pd.DataFrame() # Initialize dataframe

# Loop through all files
for f in Files:
    
    # Files have different encodings
    # Find the encoding for each file and use that in read_csv command
    with open(FilePath + f, 'rb') as rawdat:
        result = chardet.detect(rawdat.read(10000))
        
    oseDFA = pd.read_csv(FilePath + f, encoding = result['encoding'])
    oseDFA['name'] = f
    
    oseDF = oseDF.append(oseDFA)